In [1]:
import geopandas as gpd
import pandas as pd
import math
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import image collections

Landsat 7 - PY6S

In [54]:
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/Landsat7")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print(ee.Date(landsat7.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2000-01-24T13:46:48
2024-01-16T11:16:49


In [45]:
print(landsat7.aggregate_histogram('WRS_PATH').getInfo())
print(landsat7.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 624, '228.0': 729, '229.0': 13}
{'61.0': 685, '62.0': 681}


In [46]:
print(landsat7.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat7.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 298, '62.0': 326}
{'61.0': 380, '62.0': 349}


Landsat 8 PY6S

In [55]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/Landsat8")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat8.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-11T13:55:54
2024-12-12T13:47:48


In [48]:
print(landsat8.aggregate_histogram('WRS_PATH').getInfo())
print(landsat8.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 364, '228.0': 379}
{'61.0': 372, '62.0': 371}


In [49]:
print(landsat8.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat8.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 180, '62.0': 184}
{'61.0': 192, '62.0': 187}


Landsat 9 PY6S

In [53]:
landsat9 = (ee.ImageCollection("projects/ee-curuai2/assets/Py6S/LD9/Landsat9")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat9.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat9.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2021-11-06T13:49:55
2024-12-27T13:54:34


In [51]:
print(landsat9.aggregate_histogram('WRS_PATH').getInfo())
print(landsat9.aggregate_histogram('WRS_ROW').getInfo())

{'227.0': 101, '228.0': 105, '229.0': 2}
{'61.0': 104, '62.0': 104}


In [52]:
print(landsat9.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat9.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

{'61.0': 50, '62.0': 51}
{'61.0': 53, '62.0': 52}


# Transform into remote sensing reflectance and sunglint correction

In [57]:
def deglint (img):
  '''#dividir imagem corrigida por pi
  Rrs_sat_ac = Rsat_ac / pi
  fazer deglint
  Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
  correção no artigo INPE CURUAI'''
  Rrs = img.divide(math.pi)
  deglint = Rrs.select(['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2'])\
    .subtract(Rrs.select('swir1'))

  return (deglint
          .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

## Padronize band names

In [58]:
name_bands = ['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2']

Landsat 7

In [59]:
#renomear bandas

ld7 = landsat7.map(lambda img: img.rename(name_bands))
display(ld7.size().getInfo())

1366

Landsat 8

In [60]:
ld8 = landsat8.map(lambda img: img.rename(name_bands))
display(ld8.size().getInfo())

743

landsat 9

In [61]:
ld9 = landsat9.map(lambda img: img.rename(name_bands))
display(ld9.size().getInfo())

208

In [62]:
merge_col = ld8.merge(ld7).merge(ld9).sort('system:time_start').map(deglint)
display(merge_col.limit(5).getInfo())

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'blue_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [4256, 2110],
     'crs': 'EPSG:32621',
     'crs_transform': [30, 0, 598620, 0, -30, -211440]},
    {'id': 'green_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [4256, 2110],
     'crs': 'EPSG:32621',
     'crs_transform': [30, 0, 598620, 0, -30, -211440]},
    {'id': 'red_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [4256, 2110],
     'crs': 'EPSG:32621',
     'crs_transform': [30, 0, 598620, 0, -30, -211440]},
    {'id': 'nir_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions': [4256, 2110],
     'crs': 'EPSG:32621',
     'crs_transform': [30, 0, 598620, 0, -30, -211440]},
    {'id': 'swir1',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'dimensions':

# Fazer mosaico por mes

In [34]:
years = ee.List.sequence(2000,2017);
months = ee.List.sequence(1,12,3);
display(months.getInfo())

[1, 4, 7, 10]

In [37]:
monthly_mosaics =  ee.ImageCollection.fromImages(
    years.map(lambda y: months.map(lambda m: merge_col.filter(ee.Filter.calendarRange(y, y, 'year'))
    .filter(ee.Filter.calendarRange(m, ee.Number(m).add(2), 'month')).median()
    .set('year', y).set('month_init', m).set('month_end', ee.Number(m).add(2)).set('system:time_start', ee.Date.fromYMD(y, m, 1)).set('timestamp', ee.Date.fromYMD(y, m, 1).format('yyyy-MM-dd'))
    )).flatten());
display(monthly_mosaics.size().getInfo())

72

In [39]:
monthly_mosaic = monthly_mosaics.map(lambda img: img.set('band_count',img.bandNames().length())).filter(ee.Filter.gt('band_count',0))
display(monthly_mosaic.size().getInfo())
display(monthly_mosaic.limit(2).getInfo())

72

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'blue_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'green_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'red_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'nir_mean',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'swir1',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]},
    {'id': 'swir2',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0

# Mask land

In [63]:
def hsvComposite (image):
    composite = image.select(['blue_mean','green_mean','red_mean']).rgbToHsv()#.clip(limits);
    hue = composite.select("hue");
    max_mask = hue.lte(0.9)
    min_mask = hue.gte(0.3)
    return image.updateMask(max_mask).updateMask(min_mask).select(['blue_mean','green_mean','red_mean','nir_mean']);

# calculate area

In [66]:
def area_calc(img):
  '''receives an image of curuai and returns water surface area in km2 within the floodplain limits
  as a property of the input image'''

  # Get a pixel area image.
  pixel_area = ee.Image.pixelArea()

  floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/varzea_alagavel')
  image = hsvComposite(img)
  img_mask = image.gt(0)

  areaImage = img_mask.multiply(pixel_area)

  area = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': floodplain.geometry(),
    'scale': 30,
    'maxPixels': 1e10
    })
  return image.set('area_km2',ee.Number(area.get('red_mean')).divide(1e6))


In [67]:
monthly_area = monthly_mosaic.map(hsvComposite).map(area_calc)

display(monthly_area.first().getInfo())

{'type': 'Image',
 'bands': [{'id': 'blue_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'green_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'red_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'nir_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'band_count': 6,
  'area_km2': 989.0847583660595,
  'system:index': '0',
  'system:time_start': {'type': 'Date', 'value': 946684800000},
  'year': 2000,
  'month_init': 1,
  'month_end': 3,
  'timestamp': '2000-01-01'}}

# Import Model and Classify Images

In [71]:
classifier = ee.Classifier.load('projects/ee-curuai2/assets/RF_GEE_regressor')
print(classifier.getInfo())

{'type': 'Classifier.load', 'id': 'projects/ee-curuai2/assets/RF_GEE_regressor'}


In [72]:
predictors = ['blue_mean', 'green_mean','red_mean', 'nir_mean']
spm_classified = monthly_area.select(predictors).map(lambda img: img.addBands(img.classify(classifier=classifier)))
display(spm_classified.size().getInfo())

72

In [77]:
display(spm_classified.first().getInfo())

{'type': 'Image',
 'bands': [{'id': 'blue_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'green_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'red_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'nir_mean',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'classification',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'band_count': 6,
  'area_km2': 989.0847583660595,
  'system:index': '0',
  'system:time_start': {'type': 'Date', 'value': 946684800000},
  'year': 2000,
  'month_init': 1,
  'month_end': 3,
  'timestamp': '2000-01

# Export as Asset: Image Collection

In [73]:
prj = landsat8.first().select('B4').projection().getInfo()
scale = landsat8.first().select('B4').projection().nominalScale().getInfo()

In [103]:
def export_img(img):
      # define YOUR assetID
    # export

    fname = ee.String(img.get('timestamp')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(img),
        description= 'ld_mosaic_'+fname,
        assetId = 'projects/ee-curuai2/assets/landsat_water_period/water_period/mosaic_'+fname,#change the properties to add in images here
        region = ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
        crs = prj['crs'],
        scale = scale,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return img

In [104]:
col_length = spm_classified.size().getInfo()
#print(col_length)
# cannot map the function because we run things in the function in bothe client and server sides so we need to make a loop
# for very big time series It is recomendable to break the series and export data in parts
# not only because of the loop but also because of how exporting to assets works in google earth engine
# and you can run into problems if too much information is exported at the same time
for i in range(0,col_length):
    #print(i)
    list = spm_classified.toList(col_length)
    img = ee.Image(list.get(i))
    export_img(img)

exporting 2000-01-01--->done
exporting 2000-04-01--->done
exporting 2000-07-01--->done
exporting 2000-10-01--->done
exporting 2001-01-01--->done
exporting 2001-04-01--->done
exporting 2001-07-01--->done
exporting 2001-10-01--->done
exporting 2002-01-01--->done
exporting 2002-04-01--->done
exporting 2002-07-01--->done
exporting 2002-10-01--->done
exporting 2003-01-01--->done
exporting 2003-04-01--->done
exporting 2003-07-01--->done
exporting 2003-10-01--->done
exporting 2004-01-01--->done
exporting 2004-04-01--->done
exporting 2004-07-01--->done
exporting 2004-10-01--->done
exporting 2005-01-01--->done
exporting 2005-04-01--->done
exporting 2005-07-01--->done
exporting 2005-10-01--->done
exporting 2006-01-01--->done
exporting 2006-04-01--->done
exporting 2006-07-01--->done
exporting 2006-10-01--->done
exporting 2007-01-01--->done
exporting 2007-04-01--->done
exporting 2007-07-01--->done
exporting 2007-10-01--->done
exporting 2008-01-01--->done
exporting 2008-04-01--->done
exporting 2008

# Visualize gif

In [97]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 768,
    "region": ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
    "framesPerSecond": 1,
    "bands": ["classification"],
    "min": 1,
    "max": 350,
    "palette": ['blue','green', 'yellow', 'orange','red'],
}

In [98]:
geemap.download_ee_video(spm_classified, video_args, '/content/classification_SPM.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /content/classification_SPM.gif


In [99]:
geemap.add_text_to_gif(
    '/content/classification_SPM.gif',
    '/content/classification_SPM_text.gif',
    xy=("3%", "5%"),
    text_sequence=spm_classified.aggregate_array('timestamp').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)

In [100]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 768,
    "region": ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
    "framesPerSecond": 1,
    "bands": ["red_mean",'green_mean','blue_mean'],
    "min": 0,
    "max": 0.07,
}

In [101]:
geemap.download_ee_video(spm_classified, video_args, '/content/water_period.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: /content/water_period.gif


In [102]:
geemap.add_text_to_gif(
    '/content/water_period.gif',
    '/content/water_period_text.gif',
    xy=("3%", "5%"),
    text_sequence=spm_classified.aggregate_array('timestamp').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)